In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)])
    
    kinds_layer = layers.LSTM(32)(layer)
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
        layers.Softmax(name='K')(layers.Dense(KINDS_SIZE)(kinds_layer))
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = keras.Model(inputs=[input_text], outputs=outputs[:-1])
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 110)      4840        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 220)      582560      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 220)      776160      bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized, 'K': train.kind }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized, 'K': valid.kind }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [ ]:
data_rabanit = load_data(['rabanit'])

In [ ]:
data_pre_modern = load_data(['pre_modern'])

In [ ]:
data_modern = load_data(validation=0.2, source=['modern'])

In [5]:
data_all = load_data(validation=0.1, source=['biblical', 'rabanit', 'pre_modern', 'modern'])

In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

In [ ]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

In [ ]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.5e-3))
# history = fit(data_modern, CircularLearningRate(1e-3, 2e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

In [14]:
history = fit(data_all, scheduler=CircularLearningRate(0.01e-4, 1e-4, 0.01e-4))

Train on 175563 samples, validate on 19507 samples
175563/175563 [==============================] - 237s 1ms/sample - loss: 0.3234 - N_loss: 0.1633 - D_loss: 0.0469 - S_loss: 0.0043 - K_loss: 0.1088 - N_accuracy: 0.9439 - D_accuracy: 0.9817 - S_accuracy: 0.9989 - K_accuracy: 0.9635 - val_loss: 0.3568 - val_N_loss: 0.1683 - val_D_loss: 0.0479 - val_S_loss: 0.0041 - val_K_loss: 0.1363 - val_N_accuracy: 0.9422 - val_D_accuracy: 0.9813 - val_S_accuracy: 0.9989 - val_K_accuracy: 0.9528


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [ ]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [10]:
model.load_weights('./checkpoints/all')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin, actual_kind] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2]),  dataset.from_categorical(prediction[3])]
    [expected_niqqud, expected_dagesh, expected_sin, expected_kind] = [data.niqqud[s], data.dagesh[s], data.sin[s], data.kind[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a, actual_kind[i])
        print('רצוי: ', e, expected_kind[i])

print_predictions(data_all[1], 6)

מצוי:  עֶצֶם, וּכְלֵי מַתְּכוֹת, וּכְלֵי עֵץ, וּכְלֵי חֶרֶס. הֲרֵי הוּא אוֹמֵר "מִכָּל-כְּלֵי-עֵץ  3
רצוי:  עֶצֶם, וּכְלֵי מַתְּכוֹת, וּכְלֵי עֵץ, וּכְלֵי חֶרֶס. הֲרֵי הוּא אוֹמֵר "מִכָּל-כְּלִי-עֵץ  1
מצוי:  קָדְשֵׁי בֶּדֶק הַבַּיִת, בֵּין קָדְשֵׁי מִזְבֵּחַ; וְאֵין נִשְׁאֲלִין עַל הַתְּמוּרָה. וּכְשֵׁם שֶׁהָאָב  1
רצוי:  קָדְשֵׁי בֶּדֶק הַבַּיִת, בֵּין קָדְשֵׁי מִזְבֵּחַ; וְאֵין נִשְׁאָלִין עַל הַתְּמוּרָה. וּכְשֵׁם שֶׁהָאָב  1
מצוי:  שְׁפִיכַת דָּמִים, יַקִּירִי, וַאֲנִי אַרְכַּב עַל סוֹסֵךָ, כְּשֵׁם שֶׁאֲנִי רוֹכֵב עָלֶיךָ. הוּא  3
רצוי:  שְׁפִיכַת דָּמִים, יַקִּירִי, וַאֲנִי אֶרְכַּב עַל סוּסְךָ, כְּשֵׁם שֶׁאֲנִי רוֹכֵב עָלֶיךָ. הוּא  3
מצוי:  בָּנֶיָהָ, שְׁתֵּי בְּנוֹתֶיהָ עַל לְחֵי-הוֹרֵד; אַךְ מִן הַתִּינוֹק אֲשֶׁר בָּעֶרֶשׁ לֹא יָכְלֶה  3
רצוי:  בָּנֶיהָ, שְׁתֵּי בְּנוֹתֶיהָ עַל לְחִי-הַוֶּרֶד; אַךְ מִן הַתִּינוֹק אֲשֶׁר בָּעֶרֶשׂ לֹא יָכְלָה  3
מצוי:  שִׁטָעוּ--הֲרֵי בֵּית דִּין חַיָּבִין לְהָבִיא קָרְבַּן חַטָּאת עַל שֶׁגְגָתָן בַּהוֹרָאָה, וְאַף  1
רצוי:  שֶׁטָּעוּ--הֲרֵי ב

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])